In [1]:
import numpy as np
import pandas as pd

# Simulation parameters
Pt = 1
sigma2dBm = -80
sigma2 = 10**(sigma2dBm/10)
GammadB = 9
Gamma = 10**(GammadB/10)
fc = 3
dv = 2
d_SR1 = 30
d_SR2 = 60
AP_Xcoordinate = 0
AP_Ycoordinate = 0
AP_Zcoordinate = 0
distances = np.linspace(0, 65, 100)
N = 1
beta = 1
M = 12
psi = np.linspace(1, 2*np.pi, M)
phi = beta * np.exp(1j*psi)
Phi = np.diag(phi)
pathloss_3GPP_NLOS = lambda x, y: 10**((30 - 10*y*np.log10(x))/10)

# Function to calculate achievable rate and distances
def calculate_parameters(d_SR):
    d_RD_list = []
    d_SD_list = []
    C_phi_list = []

    for _ in range(10000):  # Generate 10,000 samples
        # Randomly select a distance from the predefined range
        d1 = np.random.choice(distances)

        # Calculate d_SD and d_RD
        d_SD = np.sqrt(d1**2 + dv**2)
        d_RD = np.sqrt((d1 - d_SR)**2 + dv**2)

        # Compute the channel gains using the 3GPP models
        betaSR = pathloss_3GPP_NLOS(d_SR, 2.2)
        betaRD = pathloss_3GPP_NLOS(d_RD, 2.4)
        betaSD = pathloss_3GPP_NLOS(d_SD, 3.5)

        # Small-scale fading channel
        gSDLOS = np.sqrt(0.5) * (np.random.randn(N, 1) + 1j*np.random.randn(N, 1))
        gSRLOS = np.sqrt(0.5) * (np.random.randn(N, M) + 1j*np.random.randn(N, M))
        gRDLOS = np.sqrt(0.5) * (np.random.randn(1, M) + 1j*np.random.randn(1, M))

        # Channel response in time domain
        gSR = np.sqrt(betaSR) * gSRLOS
        gRD = np.sqrt(betaRD) * gRDLOS
        gSD = np.sqrt(betaSD) * gSDLOS

        # Calculate W_n(phi)
        W = np.power(np.abs(gSR @ Phi @ gRD.T + gSD), 2)

        # Calculate achievable rate
        rate = np.log2(1 + (Pt * W / (Gamma * sigma2)))
        C_phi = rate[0, 0]

        # Append values to lists
        d_RD_list.append(d_RD)
        d_SD_list.append(d_SD)
        C_phi_list.append(C_phi)

    return d_RD_list, d_SD_list, C_phi_list

# Generate dataset for d_SR1
d_RD1, d_SD1, C_phi1 = calculate_parameters(d_SR1)

# Create a DataFrame
data = {
    'd_RD': d_RD1,
    'd_SD': d_SD1,
    'C_phi': C_phi1
}
df = pd.DataFrame(data)

# Save the dataset to a CSV file
df.to_csv('dataset.csv', index=False)
print("Dataset saved to 'dataset.csv'")

Dataset saved to 'dataset.csv'


In [2]:
import pandas as pd
df = pd.read_csv('dataset.csv')
print(df.head())

        d_RD       d_SD      C_phi
0  11.538294  41.411960  27.935335
1  14.382164  15.883992  29.419463
2   4.017981  33.544524  30.961689
3  34.401620  64.374510  23.510805
4  33.746187  63.718265  21.272536
